# Set Up to Collect Data

In [1]:
import base64
import datetime
import json
import pandas as pd
import pickle
import pprint
import numpy as np
import requests
import time

In [12]:
client_key = '6LD3RvXy5mZJeB1dYcHPuzkkD'
client_secret = 'IdwAEqB1myf4E7G9gNvJffDIHw7bOSx7cGyt0Ppge7sYUQYm8H'

In [13]:
key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [14]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}

In [15]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [16]:
json_data =  response.json()

In [17]:
access_token = json_data['access_token']

# Search for Tweets with #Olympics and Cycling
Here, I have search for tweets with the following link:
https://twitter.com/search?q=%23olympics%20cycling&src=typed_query

In [18]:
max_tweets=10000
tw_limit=100# we can get only 100 tweet per call with standard search api
tweets = []
seven_days_ago_now = datetime.datetime.now() - datetime.timedelta(days=6)

In [19]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)}
search_url = base_url + '2/tweets/search/recent'

In [20]:
def get_tweets(access_token, query, max_tweets=10, tweet_limit=10):
    """Retrieve tweets from the recent search API.
        Args
        ----
        access_token (str): A valid bearer token for making Twitter API requests.
        query (str): A valid Twitter query string for filtering the search tweets.
        max_tweets (int): The maximum number of tweets to collect in total.
        tweet_limit (int): The number of maximum tweets per API request. 
    """

    page_token = None
    tweet_data = []

    search_headers = {
        'Authorization': 'Bearer {}'.format(access_token),
        'User-Agent': 'v2FullArchiveSearchPython',
    }

    # Divides the max tweets into the appropriate number of requests based on the tweet_limit.
    for i in range(max_tweets // tweet_limit - 1):
        search_parameters = {
            'query': query,
            'max_results': tweet_limit,
            'tweet.fields': 'lang,created_at,referenced_tweets,source,conversation_id'
        }

        print(f'Request {i + 1}: {query}')
        
        # If we reach the 2nd page of results, add a next_token attribute to the search parameters
        if i > 0:
            search_parameters['next_token'] = page_token

        response = requests.get(search_url, headers=search_headers, params=search_parameters)
        if response.status_code != 200:
            print(f'\tError occurred: Status Code{response.status_code}: {response.text}')
        else:
            # We need to check for a result count before doing anything futher; if we have result_count we have data
            if response.json()['meta']['result_count'] > 0:
                tweet_data.extend(response.json()['data'])
                
                # If a 'next_token' exists, then update the page token to continue pagination through results
                if 'next_token' in response.json()['meta']:
                    page_token = response.json()['meta']['next_token']
            else:
                print(f'\tNo data returned for query!')
                break
        
        print(f'\t{len(tweet_data)} total tweets gathered')

        time.sleep(1)

    return tweet_data

In [21]:
q = '#olympics cycling -is:retweet'
olympic_tweets = get_tweets(access_token, q, max_tweets, tw_limit)

Request 1: #olympics cycling -is:retweet
	99 total tweets gathered
Request 2: #olympics cycling -is:retweet
	199 total tweets gathered
Request 3: #olympics cycling -is:retweet
	299 total tweets gathered
Request 4: #olympics cycling -is:retweet
	398 total tweets gathered
Request 5: #olympics cycling -is:retweet
	497 total tweets gathered
Request 6: #olympics cycling -is:retweet
	596 total tweets gathered
Request 7: #olympics cycling -is:retweet
	696 total tweets gathered
Request 8: #olympics cycling -is:retweet
	796 total tweets gathered
Request 9: #olympics cycling -is:retweet
	896 total tweets gathered
Request 10: #olympics cycling -is:retweet
	996 total tweets gathered
Request 11: #olympics cycling -is:retweet
	1095 total tweets gathered
Request 12: #olympics cycling -is:retweet
	1195 total tweets gathered
Request 13: #olympics cycling -is:retweet
	1295 total tweets gathered
Request 14: #olympics cycling -is:retweet
	1394 total tweets gathered
Request 15: #olympics cycling -is:retwee

In [23]:
with open('../data/archive/olympic_cycling.pkl', 'wb') as f:
    pickle.dump(olympic_tweets, f)

# Get replies to tweets

In [26]:
olympic_cycling = pd.read_pickle("../data/archive/olympic_cycling.pkl")

In [29]:
conversation_ids = [ tweet['conversation_id'] for tweet in olympic_cycling ]

In [31]:
replies = []

for id in conversation_ids[0:11]:
    q = 'conversation_id:{id}'.format(id=id)
    replies.extend(get_tweets(access_token, q, max_tweets, tw_limit))

Request 1: conversation_id:1424536982101790729
	No data returned for query!
Request 1: conversation_id:1424530437586571270
	No data returned for query!
Request 1: conversation_id:1424529490466820102
	2 total tweets gathered
Request 2: conversation_id:1424529490466820102
	4 total tweets gathered
Request 3: conversation_id:1424529490466820102
	6 total tweets gathered
Request 4: conversation_id:1424529490466820102
	8 total tweets gathered
Request 5: conversation_id:1424529490466820102
	10 total tweets gathered
Request 6: conversation_id:1424529490466820102
	12 total tweets gathered
Request 7: conversation_id:1424529490466820102
	14 total tweets gathered
Request 8: conversation_id:1424529490466820102
	16 total tweets gathered
Request 9: conversation_id:1424529490466820102
	18 total tweets gathered
Request 10: conversation_id:1424529490466820102
	20 total tweets gathered
Request 11: conversation_id:1424529490466820102
	22 total tweets gathered
Request 12: conversation_id:1424529490466820102

	8 total tweets gathered
Request 9: conversation_id:1424513383001436161
	9 total tweets gathered
Request 10: conversation_id:1424513383001436161
	10 total tweets gathered
Request 11: conversation_id:1424513383001436161
	11 total tweets gathered
Request 12: conversation_id:1424513383001436161
	12 total tweets gathered
Request 13: conversation_id:1424513383001436161
	13 total tweets gathered
Request 14: conversation_id:1424513383001436161
	14 total tweets gathered
Request 15: conversation_id:1424513383001436161
	15 total tweets gathered
Request 16: conversation_id:1424513383001436161
	16 total tweets gathered
Request 17: conversation_id:1424513383001436161
	17 total tweets gathered
Request 18: conversation_id:1424513383001436161
	18 total tweets gathered
Request 19: conversation_id:1424513383001436161
	19 total tweets gathered
Request 20: conversation_id:1424513383001436161
	20 total tweets gathered
Request 21: conversation_id:1424513383001436161
	21 total tweets gathered
Request 22: con

	19 total tweets gathered
Request 20: conversation_id:1424502101669994506
	20 total tweets gathered
Request 21: conversation_id:1424502101669994506
	21 total tweets gathered
Request 22: conversation_id:1424502101669994506
	22 total tweets gathered
Request 23: conversation_id:1424502101669994506
	23 total tweets gathered
Request 24: conversation_id:1424502101669994506
	24 total tweets gathered
Request 25: conversation_id:1424502101669994506
	25 total tweets gathered
Request 26: conversation_id:1424502101669994506
	26 total tweets gathered
Request 27: conversation_id:1424502101669994506
	27 total tweets gathered
Request 28: conversation_id:1424502101669994506
	28 total tweets gathered
Request 29: conversation_id:1424502101669994506
	29 total tweets gathered
Request 30: conversation_id:1424502101669994506
	30 total tweets gathered
Request 31: conversation_id:1424502101669994506
	31 total tweets gathered
Request 32: conversation_id:1424502101669994506
	32 total tweets gathered
Request 33: 

	90 total tweets gathered
Request 31: conversation_id:1424495783160782850
	93 total tweets gathered
Request 32: conversation_id:1424495783160782850
	96 total tweets gathered
Request 33: conversation_id:1424495783160782850
	99 total tweets gathered
Request 34: conversation_id:1424495783160782850
	102 total tweets gathered
Request 35: conversation_id:1424495783160782850
	105 total tweets gathered
Request 36: conversation_id:1424495783160782850
	108 total tweets gathered
Request 37: conversation_id:1424495783160782850
	111 total tweets gathered
Request 38: conversation_id:1424495783160782850
	114 total tweets gathered
Request 39: conversation_id:1424495783160782850
	117 total tweets gathered
Request 40: conversation_id:1424495783160782850
	120 total tweets gathered
Request 41: conversation_id:1424495783160782850
	123 total tweets gathered
Request 42: conversation_id:1424495783160782850
	126 total tweets gathered
Request 43: conversation_id:1424495783160782850
	129 total tweets gathered
Re

Request 42: conversation_id:1424495076873474051
	42 total tweets gathered
Request 43: conversation_id:1424495076873474051
	43 total tweets gathered
Request 44: conversation_id:1424495076873474051
	44 total tweets gathered
Request 45: conversation_id:1424495076873474051
	45 total tweets gathered
Request 46: conversation_id:1424495076873474051
	46 total tweets gathered
Request 47: conversation_id:1424495076873474051
	47 total tweets gathered
Request 48: conversation_id:1424495076873474051
	48 total tweets gathered
Request 49: conversation_id:1424495076873474051
	Error occurred: Status Code429: Rate limit exceeded

	48 total tweets gathered
Request 50: conversation_id:1424495076873474051
	Error occurred: Status Code429: Rate limit exceeded

	48 total tweets gathered
Request 51: conversation_id:1424495076873474051
	Error occurred: Status Code429: Rate limit exceeded

	48 total tweets gathered
Request 52: conversation_id:1424495076873474051
	Error occurred: Status Code429: Rate limit exceed

KeyboardInterrupt: 

In [34]:
# with open('olympic_gynmastics_replies.pkl', 'wb') as f:
#     pickle.dump(replies, f)